In [1]:
import torch
import json
from torch import nn
import torch.utils.data as Data

d:\Anaconda\envs\tor1131py37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
texts, labels = [], []
unique_text, unique_label = [], []
for line in open('D:\project\\vscode\RNN_Proj\data\\train_data3.txt','r',encoding='utf-8'):
    json_data = json.loads(line)
    texts.append(json_data['text'])
    labels.append(json_data['labels'])
    unique_text.extend(json_data['text'])
    unique_label.extend(json_data['labels'])


In [3]:
unique_text = list(set(unique_text))
unique_label = list(set(unique_label))
print('len(unique_text),len(unique_label)', len(unique_text), len(unique_label))

len(unique_text),len(unique_label) 1810 27


In [4]:
vocab = unique_text
word2id = {v:k for k,v in enumerate(sorted(vocab))}
id2word = {k:v for k,v in enumerate(sorted(vocab))}
label2id = {v:k for k,v in enumerate(sorted(unique_label))}
id2label = {k:v for k,v in enumerate(sorted(unique_label))}
texts_id = [[word2id[v] for k,v in enumerate(text)] for text in texts]
labels_id = [[label2id[v] for k,v in enumerate(label)] for label in labels]
length_vocab = len(vocab)
length_labels = len(unique_label)

In [7]:
label2id

{'B-COU': 0,
 'B-ECA': 1,
 'B-EQU': 2,
 'B-FIL': 3,
 'B-JS': 4,
 'B-LOC': 5,
 'B-MET': 6,
 'B-NUM': 7,
 'B-ORG': 8,
 'B-PER': 9,
 'B-POS': 10,
 'B-SJ': 11,
 'B-TIM': 12,
 'I-COU': 13,
 'I-ECA': 14,
 'I-EQU': 15,
 'I-FIL': 16,
 'I-JS': 17,
 'I-LOC': 18,
 'I-MET': 19,
 'I-NUM': 20,
 'I-ORG': 21,
 'I-PER': 22,
 'I-POS': 23,
 'I-SJ': 24,
 'I-TIM': 25,
 'O': 26}

In [69]:
in_dim = 512
hidden_dim = length_labels
epoch = 10
batch_size = 1

In [51]:
len(texts)

940

In [45]:
Embedding = nn.Embedding(length_vocab, in_dim)
texts_id_tensor = torch.tensor(texts_id, dtype=float)
texts_embedding = Embedding(texts_id_tensor)

ValueError: expected sequence of length 21 at dim 1 (got 44)

In [52]:
class MyDataSet(Data.Dataset):
    def __init__(self, texts, labels):
        super(MyDataSet, self).__init__()
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, index):
        return self.texts[index], self.labels[index]
loader = Data.DataLoader(MyDataSet(texts=texts_id, labels=labels_id), batch_size=1)

In [53]:
# RNNCell
class RNN_Cell(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        """
        in_dim: 4 输入字符的向量的维度
        hidden_dim: 4 隐层维度
        """
        super(RNN_Cell, self).__init__()
        self.Wx = nn.Linear(in_dim, hidden_dim)
        self.Wh = nn.Linear(hidden_dim, hidden_dim)
        # self.b = nn.Parameter(1, hidden_dim)

    def forward(self, x, h_1):
        h = torch.tanh(self.Wx(x) + self.Wh(h_1))
        return h

class RNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, batch_size):
        """
        input_size: 4
        hidden_size: 4
        batch_size: 1
        """
        super(RNN, self).__init__()
        self.batch_size = batch_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        # self.rnncell = torch.nn.RNNCell(input_size=self.input_size, hidden_size=self.hidden_size)
        self.rnncell = RNN_Cell(in_dim=self.input_size, hidden_dim=self.hidden_size)

    def forward(self, input, hidden):
        hidden = self.rnncell(input, hidden)
        return hidden

    def init_hidden(self):
        return torch.zeros(self.batch_size, self.hidden_size)

In [70]:
Embedding = nn.Embedding(length_vocab, in_dim)

net = RNN(input_size=in_dim, hidden_size=hidden_dim, batch_size=batch_size)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)

for i in range(epoch):
    for texts_id, labels_id in loader:
        texts_embedding = Embedding(torch.tensor(texts_id)).view(-1, batch_size, in_dim)
        labels_y = torch.tensor(labels_id).view(-1, 1)

        loss = 0
        optimizer.zero_grad()
        hidden = net.init_hidden()
        print('Predicted string: ', end='')
        for input, label in zip(texts_embedding, labels_y):  # inputs：seg_len * batch_size * input_size；labels：
            hidden = net.forward(input, hidden)
            loss += criterion(hidden, label)  # 要把每个字母的loss累加    =([1,4], [1])
            _, idx = hidden.max(dim=1)
            # 输出预测
            print(id2label[idx.item()]+' ', end='')
        loss.backward()
        optimizer.step()
        print(', Epoch [%d/%d] loss=%.4f' % (i+1,epoch, loss.item())) 

Predicted string: B-SJB-COUB-METI-SJB-ORGI-FILB-ECAB-SJI-COUI-COUI-LOCI-PERB-COUI-ECAI-POSB-ECAB-PERB-TIMB-FILI-TIMI-COU, Epoch [1/10] loss=73.9828
Predicted string: B-METB-EQUI-TIMI-JSI-PERI-POSI-FILB-COUB-JSI-ECAB-FILI-EQUB-EQUI-SJB-EQUB-LOCI-POSI-JSB-ORGB-JSB-ORGB-FILI-SJI-ORGB-ORGI-FILB-POSB-TIMB-ECAB-ORGI-LOCB-FILB-METB-PERB-LOCB-ECAB-JSB-JSB-FILB-PERI-EQUB-ORGB-TIMI-EQU, Epoch [1/10] loss=159.3264
Predicted string: I-TIMOB-TIMI-SJB-ORGI-EQUB-FILI-ECAI-METB-EQUB-SJI-ORGB-LOCB-POSI-PERI-TIMB-COUI-SJB-LOCI-SJI-ORGI-POSI-ECAB-COUI-EQUB-POSI-COUI-EQUB-METI-ECAB-COUI-EQUB-LOCI-NUMB-POSOI-PEROB-COUB-FIL, Epoch [1/10] loss=136.2150
Predicted string: B-SJB-EQUI-TIMB-EQUB-ECAB-ECAB-PERB-LOCI-SJI-PERB-ORGI-NUMB-FILB-ORGB-JSB-JSB-JSB-ORGB-JSI-TIMB-LOCI-TIMB-LOCB-SJI-EQUB-ECAI-TIMOI-SJB-EQUI-METB-ECAB-ECAB-PERI-EQUB-PERB-TIMI-ECAI-JSB-PERB-LOCB-POSB-EQUI-EQUI-POSI-ECAI-SJB-LOCB-JSB-PERB-COUB-LOCI-FILI-LOCI-TIMB-COUB-COUB-TIMOB-TIM, Epoch [1/10] loss=211.9784
Predicted string: I-SJI-POSI-ECAB-

In [ ]:
net = RNN(input_size=in_dim, hidden_size=hidden_dim, batch_size=batch_size)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)

epoch = 5500
for i in range(epoch):
    loss = 0
    optimizer.zero_grad()
    hidden = net.init_hidden()
    print('Predicted string: ', end='')
    for input, label in zip(inputs, labels):  # inputs：seg_len * batch_size * input_size；labels：
        hidden = net.forward(input, hidden)
        loss += criterion(hidden, label)  # 要把每个字母的loss累加    =([1,4], [1])
        _, idx = hidden.max(dim=1)
        # 输出预测
        print(idx2char[idx.item()], end='')
    loss.backward()
    optimizer.step()
    print(', Epoch [%d/%d] loss=%.4f' % (i+1,epoch, loss.item()))

In [67]:
e = nn.Embedding(length_vocab, in_dim)
em = e(torch.tensor([1,2,3]))
print(em)

tensor([[ 0.6928, -0.2705,  0.5378,  ..., -0.7184,  0.1899, -0.2854],
        [ 0.2923, -1.0592,  0.8053,  ..., -0.2046, -0.4528, -0.0522],
        [-0.5720, -0.6474,  0.6498,  ..., -1.3919,  0.2425, -0.0108]],
       grad_fn=<EmbeddingBackward0>)


In [63]:
em.shape

torch.Size([3, 512])

In [17]:
a = torch.tensor([[1,9,3,4]])
a.max(dim=1)

torch.return_types.max(
values=tensor([9]),
indices=tensor([1]))

In [12]:
a = torch.tensor([[1,9,3,4], [1,4,6,4], [2,9,3,4]])
b = a[torch.tensor([torch.tensor([1]), torch.tensor([2])])]

In [15]:
b

tensor([[1, 4, 6, 4],
        [2, 9, 3, 4]])